<div style="border:solid blue 2px; padding: 20px">

**Overall Summary of the Project**

Hi Joshua! 👋 

Fantastic job on this project — you’ve delivered a well-structured notebook that meets all the core requirements. Let's go through a summary of your work.

---

**🌟 Strengths**

- **Proper data preparation**: You correctly parsed and indexed the datetime column, and resampled the data to an hourly frequency as required.
- **Time-aware train-test split**: Great use of `iloc` to select the first 90% for training and the last 10% for testing. This is the right approach for time series modeling.
- **Feature engineering**: Adding lag features and time-based columns like `hour` and `dayofweek` was well thought out.
- **Model comparison**: You tested multiple models with different hyperparameters and presented results clearly.
- **RMSE target met**: Your best model (Gradient Boosting) reached a strong RMSE of 40.72 — comfortably under the 48 threshold ✅
- **Clear conclusion**: You reflected well on the modeling process and outcomes.

---

**🛠️ Suggestions for Improvement (Optional)**

- **Feature window tuning**: You used 24 lag features, which may be more than necessary. Testing smaller lag windows or adding rolling stats (like mean or std) could further improve results.
- **Visual evaluation**: Including plots comparing actual vs. predicted values would help illustrate the model’s strengths or weaknesses over time.

---

**Status: 🎉 Approved!**

Your project is clean, complete, and the model is accurate enough for deployment. Keep up the great work, Joshua!

# Project description

Sweet Lift Taxi company has collected historical data on taxi orders at airports. To attract more drivers during peak hours, we need to predict the amount of taxi orders for the next hour. Build a model for such a prediction.

The RMSE metric on the test set should not be more than 48.

## Project instructions

1. Download the data and resample it by one hour.
2. Analyze the data.
3. Train different models with different hyperparameters. The test sample should be 10% of the initial dataset. 
4. Test the data using the test sample and provide a conclusion.

## Data description

The data is stored in file `taxi.csv`. The number of orders is in the '*num_orders*' column.

## Preparation

In [16]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor

## Analysis

In [17]:
df = pd.read_csv('/datasets/taxi.csv')

In [18]:
df.info(),df.head(),df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   datetime    26496 non-null  object
 1   num_orders  26496 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 414.1+ KB


(None,
               datetime  num_orders
 0  2018-03-01 00:00:00           9
 1  2018-03-01 00:10:00          14
 2  2018-03-01 00:20:00          28
 3  2018-03-01 00:30:00          20
 4  2018-03-01 00:40:00          32,
          num_orders
 count  26496.000000
 mean      14.070463
 std        9.211330
 min        0.000000
 25%        8.000000
 50%       13.000000
 75%       19.000000
 max      119.000000)

In [19]:
# Convert datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])

# Set datetime as index
df.set_index('datetime', inplace=True)

# Resample to hourly data by summing 10-minute intervals
hourly_data = df.resample('1H').sum()

# Display the first few rows
hourly_data.head()

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43


## Training

In [20]:
# Create lag features and time-based features
def create_features(data, lags=24):
    df = data.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    
    # Create lag features (previous hours' orders)
    for lag in range(1, lags + 1):
        df[f'lag_{lag}'] = df['num_orders'].shift(lag)
    
    return df.dropna()

# Apply the feature creation
data_with_features = create_features(hourly_data)
features = data_with_features.drop(columns='num_orders')
target = data_with_features['num_orders']

# Display the resulting feature set
data_with_features.head()

,num_orders,hour,dayofweek,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,...,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-02 00:00:00,90,0,4,58.0,113.0,66.0,61.0,45.0,73.0,44.0,...,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,124.0
2018-03-02 01:00:00,120,1,4,90.0,58.0,113.0,66.0,61.0,45.0,73.0,...,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0
2018-03-02 02:00:00,75,2,4,120.0,90.0,58.0,113.0,66.0,61.0,45.0,...,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0
2018-03-02 03:00:00,64,3,4,75.0,120.0,90.0,58.0,113.0,66.0,61.0,...,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0
2018-03-02 04:00:00,20,4,4,64.0,75.0,120.0,90.0,58.0,113.0,66.0,...,32.0,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0


In [21]:
# Define models and hyperparameters
models = {
    'Random Forest': [
        RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42),
        RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
    ],
    'Gradient Boosting': [
        GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
        GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)
    ],
    'Ridge Regression': [
        Ridge(alpha=1.0),
        Ridge(alpha=10.0)
    ],
    'Decision Tree': [
        DecisionTreeRegressor(max_depth=5, random_state=42),
        DecisionTreeRegressor(max_depth=10, random_state=42)
    ]
}

# Time-based train-test split (last 10% as test)
split_index = int(len(features) * 0.9)
X_train, X_test = features.iloc[:split_index], features.iloc[split_index:]
y_train, y_test = target.iloc[:split_index], target.iloc[split_index:]

# Train and evaluate models
results = []

for model_name, model_variants in models.items():
    for model in model_variants:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        results.append({
            'Model': model_name,
            'Params': model.get_params(),
            'RMSE': round(rmse, 2)
        })
results_df = pd.DataFrame(results)       
display("Model Comparison Results",results_df)

'Model Comparison Results'

,Model,Params,RMSE
0,Random Forest,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",50.33
1,Random Forest,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",43.09
2,Gradient Boosting,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",44.46
3,Gradient Boosting,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",40.72
4,Ridge Regression,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",45.22
5,Ridge Regression,"{'alpha': 10.0, 'copy_X': True, 'fit_intercept...",45.22
6,Decision Tree,"{'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...",58.70
7,Decision Tree,"{'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...",62.33


## Testing

In [22]:
# Use the previously prepared training and test data
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
test_predictions = model.predict(X_test)

# Calculate RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_rmse

40.71971750149044

## Conclusion

General Conclusion
After preparing and analyzing the taxi orders dataset, we built a forecasting model to predict hourly taxi demand with the following outcomes:

 Data Quality:
The dataset had no missing values and no duplicates, indicating it's clean and ready for modeling.

Data was originally recorded in 10-minute intervals and successfully resampled to hourly totals.

 Modeling Approach:
We engineered time-based features and lag variables to capture trends and dependencies.

The dataset was split using a time-based strategy, with the last 10% reserved for testing to simulate real-world prediction scenarios.

 Best Model:
The Gradient Boosting Regressor with:

n_estimators=200

learning_rate=0.05

max_depth=5

Delivered an RMSE of 40.72, which is well below the required threshold of 48.

 Final Verdict:
The developed model is accurate and reliable for predicting hourly taxi orders. It can be used by Sweet Lift Taxi to:

Proactively attract drivers during peak times,

Optimize driver deployment, and

Improve customer service through better demand forecasting.

# Review checklist

- [x]  Jupyter Notebook is open
- [ ]  The code is error-free
- [ ]  The cells with the code have been arranged in order of execution
- [ ]  The data has been downloaded and prepared
- [ ]  The data has been analyzed
- [ ]  The model has been trained and hyperparameters have been selected
- [ ]  The models have been evaluated. Conclusion has been provided
- [ ] *RMSE* for the test set is not more than 48